# 使用Word2Vec 經過代碼化&向量化產生文章摘要

In [1]:
!pip install gensim==3.8.3

Defaulting to user installation because normal site-packages is not writeable


## 下載最新的中文WIKI文章包

In [2]:
!pip install wget

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import wget
from pathlib import Path

In [4]:
wikiLink = 'https://dumps.wikimedia.org/zhwiki/20220820/zhwiki-20220820-pages-articles-multistream.xml.bz2'
filePath = Path('zhwiki-20220820-pages-articles-multistream.xml.bz2')
if filePath.is_file():
  fileName = filePath
  print(fileName)
else:
  fileName = wget.download(wikiLink, bar=wget.bar_thermometer)
  print(fileName)

zhwiki-20220820-pages-articles-multistream.xml.bz2


## 使用 WikiCorpus 清理內容

In [5]:
from gensim.corpora import WikiCorpus
wikiCorpus = WikiCorpus(fileName, dictionary={})
text_num = 0
if Path('wiki_text.txt') != True:
  with open('wiki_text.txt', 'w', encoding='utf-8') as f:
    for text in wikiCorpus.get_texts():
      f.write(' '.join(text)+'\n')
      text_num += 1
      if text_num % 10000 == 0:
        print('{} articles processed.'.format(text_num))
      if text_num >10000:
        break
    print('{} articles processed.'.format(text_num))

10000 articles processed.
10001 articles processed.


## 斷詞 (使用Jieba)

In [6]:
!pip install jieba
!pip install opencc

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 4.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
Using legacy 'setup.py install' for jieba, since package 'wheel' is not installed.
  Running setup.py install for jieba ... done
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.1/770.1 kB 2.5 MB/s eta 0:00:00a 0:00:01


In [7]:
import jieba
from opencc import OpenCC

cc = OpenCC('s2t') # 簡體轉繁體
if Path('wiki_text_seg.txt') != True:
  with open('wiki_text_seg.txt', 'w', encoding='utf-8') as new_f:
    with open('wiki_text.txt', 'r', encoding='utf-8') as f:
      for times, data in enumerate(f, 1):
        print('data num:', times)
        data = cc.convert(data)
        data = jieba.cut(data)
        data = [word for word in data if word != ' ']
        data = ' '.join(data)

        new_f.write(data)

Building prefix dict from the default dictionary ...


data num: 1


Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.602 seconds.
Prefix dict has been built successfully.


data num: 2
data num: 3
data num: 4
data num: 5
data num: 6
data num: 7
data num: 8
data num: 9
data num: 10
data num: 11
data num: 12
data num: 13
data num: 14
data num: 15
data num: 16
data num: 17
data num: 18
data num: 19
data num: 20
data num: 21
data num: 22
data num: 23
data num: 24
data num: 25
data num: 26
data num: 27
data num: 28
data num: 29
data num: 30
data num: 31
data num: 32
data num: 33
data num: 34
data num: 35
data num: 36
data num: 37
data num: 38
data num: 39
data num: 40
data num: 41
data num: 42
data num: 43
data num: 44
data num: 45
data num: 46
data num: 47
data num: 48
data num: 49
data num: 50
data num: 51
data num: 52
data num: 53
data num: 54
data num: 55
data num: 56
data num: 57
data num: 58
data num: 59
data num: 60
data num: 61
data num: 62
data num: 63
data num: 64
data num: 65
data num: 66
data num: 67
data num: 68
data num: 69
data num: 70
data num: 71
data num: 72
data num: 73
data num: 74
data num: 75
data num: 76
data num: 77
data num: 78
data nu

## 使用Gensim 訓練word2vec模型

In [12]:
from gensim.models import word2vec
train_data = word2vec.LineSentence('wiki_text_seg.txt')
model = word2vec.Word2Vec(train_data)

model.save('word2vec.model')

## 測試模型效果

In [13]:
model = word2vec.Word2Vec.load('word2vec.model')
print(model.wv['生物'].shape)

for item in model.wv.most_similar('生物'):
  print(item)

(100,)
('微生物', 0.8435463905334473)
('遺傳學', 0.7710638642311096)
('真核', 0.765525221824646)
('細菌', 0.7594730854034424)
('植物', 0.7535208463668823)
('古菌', 0.7472869157791138)
('基因', 0.7346128225326538)
('原核', 0.7327805757522583)
('靈長', 0.7305771112442017)
('胚胎', 0.7282221913337708)


## word2vec 文章摘要

In [64]:
from gensim.summarization import summarize
news="""
2015年有不少台灣媒體前往柬埔寨採訪陳豐天，隔年皇家集團即來台推銷建案，吸引台灣人前往柬埔寨投資。不過，2019年2月皇家一號風光動工不到10 天，柬埔寨總理洪森大規模掃毒，陳豐天的ROCK俱樂部被查出50公斤毒品，當時柬國警方還高調宣布這是史上最大規模的掃毒行動，共逮捕陳豐天等300餘人。
後來陳豐天遭判刑4年，但他以罹患肺病為由要求住院治療，直到去年5月當地法院才准他獲釋，之後陳豐天即不再如以往高調過活及招商。本刊調查，皇家集團除了2016年推出的皇家一號建案出狀況，早先在2015年推出的「皇家First 1」也爆出糾紛。
2018年有2位名民眾向本刊投訴，2015年以總價約新台幣250萬元各買下一戶位於首都金邊的預售屋皇家First 1，預計2018年6月交屋，但後來建案停工數月，負責台灣代銷的全球國際地產早已人去樓空。
皇家First 1投資人求助無門，得知皇家集團在台推出皇家一號，於是轉而找上甲桂林集團請求協助，但因為這兩個建案在台是由不同的代銷業者處理，建商、代銷、業務互踢皮球，中間還傳出承辦人員過世，全案因此不了了之。
當年購買皇家First 1遇到的問題至今無解，又傳出有台灣人購買皇家一號預售案苦等不到完工，由於當年是由甲桂林創辦人曹瑞濱開記者會代銷，但合約是與柬埔寨皇家集團簽訂，打跨國官司難度非常高，投訴人僅能寄望甲桂林集團協助解約拿回款項或求償，不要讓他們成為海外不動產投資的孤兒。
"""
print(news)


2015年有不少台灣媒體前往柬埔寨採訪陳豐天，隔年皇家集團即來台推銷建案，吸引台灣人前往柬埔寨投資。不過，2019年2月皇家一號風光動工不到10 天，柬埔寨總理洪森大規模掃毒，陳豐天的ROCK俱樂部被查出50公斤毒品，當時柬國警方還高調宣布這是史上最大規模的掃毒行動，共逮捕陳豐天等300餘人。
後來陳豐天遭判刑4年，但他以罹患肺病為由要求住院治療，直到去年5月當地法院才准他獲釋，之後陳豐天即不再如以往高調過活及招商。本刊調查，皇家集團除了2016年推出的皇家一號建案出狀況，早先在2015年推出的「皇家First 1」也爆出糾紛。
2018年有2位名民眾向本刊投訴，2015年以總價約新台幣250萬元各買下一戶位於首都金邊的預售屋皇家First 1，預計2018年6月交屋，但後來建案停工數月，負責台灣代銷的全球國際地產早已人去樓空。
皇家First 1投資人求助無門，得知皇家集團在台推出皇家一號，於是轉而找上甲桂林集團請求協助，但因為這兩個建案在台是由不同的代銷業者處理，建商、代銷、業務互踢皮球，中間還傳出承辦人員過世，全案因此不了了之。
當年購買皇家First 1遇到的問題至今無解，又傳出有台灣人購買皇家一號預售案苦等不到完工，由於當年是由甲桂林創辦人曹瑞濱開記者會代銷，但合約是與柬埔寨皇家集團簽訂，打跨國官司難度非常高，投訴人僅能寄望甲桂林集團協助解約拿回款項或求償，不要讓他們成為海外不動產投資的孤兒。



In [65]:
seq = [word for word in jieba.cut(news) if word != ' ']
seq = ' '.join(seq)
seq

'\n 2015 年 有 不少台 灣媒體 前往 柬埔寨 採訪 陳 豐天 ， 隔年 皇家 集團 即 來 台 推銷 建案 ， 吸引 台灣 人 前往 柬埔寨 投資 。 不過 ， 2019 年 2 月 皇家 一號 風光動工 不到 10 天 ， 柬埔寨 總理 洪森 大規模 掃毒 ， 陳 豐天 的 ROCK 俱樂部 被 查出 50 公斤 毒品 ， 當時 柬國 警方 還高調 宣布 這是 史上 最大 規模 的 掃毒 行動 ， 共 逮捕 陳 豐天 等 300 餘人 。 \n 後 來 陳 豐天 遭 判刑 4 年 ， 但 他 以 罹患 肺病 為 由 要求 住院 治療 ， 直到 去年 5 月 當地 法院 才准 他 獲釋 ， 之後陳 豐天 即 不再 如 以往 高調 過活 及 招商 。 本刊 調查 ， 皇家 集團 除了 2016 年 推出 的 皇家 一號 建案 出狀況 ， 早先 在 2015 年 推出 的 「 皇家 First 1 」 也 爆出 糾紛 。 \n 2018 年 有 2 位名民眾 向 本刊 投訴 ， 2015 年 以 總價 約 新 台幣 250 萬元 各買下 一戶位 於 首都 金邊 的 預售屋 皇家 First 1 ， 預計 2018 年 6 月 交屋 ， 但後來 建案 停工 數月 ， 負責 台灣代銷 的 全球 國際 地產 早已 人去 樓空 。 \n 皇家 First 1 投資人 求助 無門 ， 得知 皇家 集團 在 台 推出 皇家 一號 ， 於 是 轉而 找 上 甲 桂林 集團 請求 協助 ， 但 因為 這兩個 建案 在 台 是 由 不同 的 代銷業者 處理 ， 建商 、 代銷 、 業務互 踢皮球 ， 中間 還傳 出承辦 人員過 世 ， 全案 因此 不了了之 。 \n 當年 購買 皇家 First 1 遇到 的 問題 至今 無解 ， 又 傳出 有 台灣 人購 買 皇家 一號 預售案 苦 等 不到 完工 ， 由 於 當年 是 由 甲 桂林 創辦 人 曹瑞濱 開記者 會代銷 ， 但合約 是 與 柬埔寨 皇家 集團 簽訂 ， 打 跨國 官司 難度 非常 高 ， 投訴 人僅 能 寄望 甲 桂林 集團 協助 解約 拿 回款 項 或 求償 ， 不要 讓 他們 成為 海外 不動產 投資 的 孤兒 。 \n'

In [66]:
summarize(seq).replace(' ', '')

'2018年有2位名民眾向本刊投訴，2015年以總價約新台幣250萬元各買下一戶位於首都金邊的預售屋皇家First1，預計2018年6月交屋，但後來建案停工數月，負責台灣代銷的全球國際地產早已人去樓空。'